In [91]:
import tweepy

In [96]:
CONSUMER_KEY = CONSUMER_KEY
CONSUMER_SECRET = CONSUMER_SECRET
ACCESS_KEY = ACCESS_KEY
ACCESS_SECRET = ACCESS_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print('Authentication OK')
except:
    print('Error during authentication')

Authentication OK


In [97]:
import pandas as pd

## Twitter bot will reply to the tweet

In [99]:
def twitter_reply(status_id):
    reply_text = '''
    ------- test -------
    We noticed that you are repoting a police incident. 
    '''

    api.update_status(status = reply_text, in_reply_to_status_id=status_id)
    return 'done!'

twitter_reply(1395143654331744256)

'done!'

In [54]:
df_data = []
for tweet in api.search(q='Joan Villar', lang='en', rpp=5)[:5]:
    test_data = [tweet.user.screen_name, tweet.id, tweet.text]
    df_data.append(test_data)
df = pd.DataFrame(data=df_data, columns=['username', 'id', 'tweet'])
df

,username,id,tweet
0,JoanVillar14,1395144910580314112,Joan Villar epic student
1,JoanVillar14,1395143654331744256,Joan Villar test for tweepy
2,alvinSeven77,1394786090175795200,@AJemIlagan She has St. Joan of Arc's spirit b...


## Twitter bot will reply to the user through DM

In [80]:
def direct_message(username):
    user = api.get_user(username)
    message = '''
    Just testing some features.
    This sends a direct message to the person
    '''
    api.send_direct_message(user.id, message)
    return 'direct message sent'

direct_message('@JoanVillar14')

'direct message sent'

Improvements:

1. Depending on the reply, gather more information such as:
- did it really happen?
- where did it happen

2. They pin the place of incident on a map
- If tweet's geolocation is on, get the coordinates

-------


* There are a few things to keep in mind in designing a model that replies to specific replies. This might be another project in the future.
* Since the incidents are already identified, it might be best to just get the location from the user.